In [1]:
## comparing the arm estimate and the value from the imu

import numpy as np
import collections

from pinocchio.visualize import MeshcatVisualizer as Visualizer
import time

from matplotlib import pyplot as plt
from dataclasses import dataclass

from arm_model import create_arm

from imu_measurements import ArmMeasurement
from estimation_problem import solve_estimation_problem
from interface import ExoSkeletonUDPInterface


In [2]:
interface = ExoSkeletonUDPInterface()

In [12]:
rmodel, rdata, gmodel, cmodel = create_arm()

viz = Visualizer(rmodel, gmodel, cmodel)
viz.initViewer(open=False)
viz.loadViewerModel()
viz.initializeFrames()
viz.display_frames = True
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
counter = 0
interface.calibrate()

calibration ...
setting offset ...
finished calibration ...


In [ ]:
@dataclass
class ArmMeasurementCurrent:
    arm_orientation : np.array
    palm_orientation : np.array

imus = ArmMeasurement("/dev/ttyACM0", "/dev/ttyACM1", "/dev/ttyACM2")
estimate_x0 = np.zeros(rmodel.nq + rmodel.nv)

T = 30
measurement = collections.deque(maxlen=T)
counter = 0

print("reading measurements")
while True:
    imus.update_measurements()  
    measurement.append(ArmMeasurementCurrent(imus.arm_orientation, imus.palm_orientation))
    # interface.setCommand([0], [0.], [0], [0], [6.0])

    if counter > T:
        estimate = solve_estimation_problem(measurement, T, rmodel, estimate_x0)
        viz.display(estimate.xs[-1][:rmodel.nq])
        estimate_x0 = estimate.xs[-1]
        interface.setCommand([0], [0.], [0], [0], [0.0])
    
    # time.sleep(0.01)
    counter += 1

del imus

In [6]:
while True:
    interface.setCommand([0], [0.], [0], [0], [0.0])
    time.sleep(0.001)
    state = interface.getState()
    # print(state["motor_q"], state["q"])

KeyboardInterrupt: 